In [1]:
# import libraries
import os
import numpy as np
from tqdm import tqdm

from ifum_stitch import Stitch
from ifum_maskopt import Mask
from ifum_rectify import Rectify
from ifum_calibrate import Calibrate
from ifum_stack import Stack

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# directory containing unprocessed files
directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240212\\"

# all files included in a single stack, repeat where necessary
# only include string in file that includes all files from single exposure
data_filenames = ["1156","1157","1158","1162","1163","1164","1168","1169","1170"]
arc_filenames = ["1161","1161","1161","1167","1167","1167","1172","1172","1172"]
flat_filenames = ["1160","1160","1160","1166","1166","1166","1171","1171","1171"]

# mask???
# mode LR,STD,HR
mode = "STD"

# bad masks (on scale 1-276)
bad_blues = [23]
bad_reds = []

# stars to use in WCS (list RA,Dec)
# all stars should be present in at least some dithers
wcs_stars = [[74.8322, -58.6579],
             [74.8279, -58.6548],
             [74.8314, -58.6526],
             [74.8254, -58.6572],
             [74.8303, -58.6543]]

# plot or not
plot = False

In [3]:
# assert statements!!!
bad_masks = [np.array(bad_blues)-1,np.array(bad_reds)-1]
wcs_stars = np.array(wcs_stars)
if mode == "STD":
    total_masks = 552
    mask_groups = 12
else:
    print("invalid mode")

<h1><strong><span style="color:purple">STITCH</h1>

<span style="color:orange"><strong>creates 2 files, one for each detector, for both the data and arc files. also produces a cosmic ray mask for the data.

In [4]:
# out directory where all files are stored
if not os.path.exists("out"):
    os.makedirs("out")

# stitch and create file
for file in tqdm(data_filenames+arc_filenames+flat_filenames):
    file_to_stitch = Stitch(directory,file,None,None,None,None,None)
    file_to_stitch.load_files()
    file_to_stitch.save_file()
print("stitched files saved")

100%|██████████| 27/27 [00:32<00:00,  1.21s/it]

stitched files saved


In [5]:
# use flat and median filter to get rid of internal bias
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_bias = Stitch(directory,None,None,"b",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
    file_for_bias = Stitch(directory,None,None,"r",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
print("internal bias solved")

100%|██████████| 9/9 [00:47<00:00,  5.30s/it]

internal bias solved


In [6]:
# create cosmic ray masks
for datafilename in tqdm(data_filenames):
    file_for_cmray = Stitch(directory,None,None,"b",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
    file_for_cmray = Stitch(directory,None,None,"r",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
print("cosmic ray masks created")

100%|██████████| 9/9 [15:00<00:00, 100.04s/it]

cosmic ray masks created


<h1><strong><span style="color:purple">OPTIMIZE MASK</h1>

<span style="color:orange"><strong>use current mask for single gaussian fits to get better mask guess  

In [ ]:
# first guess; complex guess
# IMPLEMENT RANSAC??? (HBDSCAN?)
for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_maskopt.py:128: OptimizeWarning: Covariance of the parameters could not be estimated
  continuum,_ = scipy.optimize.curve_fit(self.f_2,cutoffs[0::6],flat_data[cutoffs[0::6],x])
100%|██████████| 3/3 [32:09<00:00, 643.04s/it]


In [4]:
center_deg = 5
sigma_deg = 3

for flatfilename in np.unique(flat_filenames):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)    

In [9]:
sig_mult = 1.5

for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)

100%|██████████| 3/3 [05:47<00:00, 115.79s/it]


In [4]:
# optimize arc files
expected_peaks = 15
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks)

100%|██████████| 3/3 [28:27<00:00, 569.08s/it]


In [5]:
# optimize data files
expected_peaks = 25
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks)
    file_for_mask.get_rots(arcfilename,datafilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks)
    file_for_mask.get_rots(arcfilename,datafilename)

  0%|          | 0/9 [00:00<?, ?it/s]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_maskopt.py:357: OptimizeWarning: Covariance of the parameters could not be estimated
  popt,_ = scipy.optimize.curve_fit(ifum_utils.gauss,x[mask_area],intensities[i][mask_area],p0=p0)
 11%|█         | 1/9 [16:00<2:08:03, 960.38s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils

In [6]:
# create masks for data
sig_mult = 1.5

for datafilename, flatfilename in tqdm(zip(data_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename)

100%|██████████| 9/9 [58:07<00:00, 387.47s/it]  


In [7]:
# create masks for arc!
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)

100%|██████████| 3/3 [06:48<00:00, 136.13s/it]


<h1><strong><span style="color:purple">RECTIFY</h1>

<span style="color:orange"><strong>rectify + calibrate

In [10]:
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

  0%|          | 0/9 [00:00<?, ?it/s]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
 11%|█         | 1/9 [01:58<15:51, 118.99s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
 22%|██▏       | 2/9 [03:54<13:38, 116.95s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectr

In [4]:
# calibrate (rectified xs to wls)
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.calib()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.calib()

  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [00:27<03:37, 27.14s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_rectify.py:193: OptimizeWarning: Covariance of the parameters could not be estimated
  popt,pcov = scipy.optimize.curve_fit(ifum_utils.gauss,data_xs[m-1][mask_area],data_intensities[m-1][mask_area],p0=p0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_rectify.py:228: OptimizeWarning: Covariance of the parameters could not be estimated
  popt,pcov = scipy.optimize.curve_fit(ifum_utils.gauss,arc_xs[m-1][mask_area],arc_intensities[m-1][mask_area],p0=p0)
100%|██████████| 9/9 [03:40<00:00, 24.52s/it]


<h1><strong><span style="color:purple">CALIBRATE</h1>

<span style="color:orange"><strong>calibrate

In [4]:
sig_mult = 1.5
bins = np.arange(7500,10000,1)

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.get_spectra(sig_mult,bins,color="b")
    file_for_calib.get_spectra(sig_mult,bins,color="r")

100%|██████████| 9/9 [3:28:17<00:00, 1388.62s/it]  


In [4]:
# intenisty calibrate all spectra, combine blue and red amplifiers

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.intensity_corr()

  0%|          | 0/9 [00:00<?, ?it/s]

c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_calibrate.py:128: OptimizeWarning: Covariance of the parameters could not be estimated
  popt,pcov = scipy.optimize.curve_fit(self.gauss_background,
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_calibrate.py:181: RuntimeWarning: divide by zero encountered in divide
  ratio = avg_sky/sky_int
 11%|█         | 1/9 [00:34<04:33, 34.14s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:359: RankWarning: Polyfit may be poorly conditioned
  fit = np.polyfit(x[polymask],y[polymask],deg,w=weight[polymask])
 33%|███▎      | 3/9 [01:56<03:58, 39.73s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:359: RankWarning: Polyfit may be poorly conditioned
  fit = np.polyfit(x[polymask],y[polymask],deg,w=weight[polymask])
c:\Users\daniel\OneDrive - The Unive

In [5]:
# ONLY VIZ

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

 11%|█         | 1/9 [00:27<03:41, 27.69s/it]


KeyboardInterrupt: 

<h1><strong><span style="color:purple">STACK</h1>

<span style="color:orange"><strong>stack dithers

In [5]:
# individually create datacubes to be used in dither stack

files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.hex_to_grid()
files_for_stack.spectra_to_datacube()

hexagon grid: 99.000,88.777
pixel grid: 99,89
8811 pixels


overlap percentages: 100%|██████████| 8811/8811 [00:37<00:00, 233.53it/s]


8691 (98.638%) of pixels overlap with hexagon grid
8183 (92.873%) of pixels overlap fully with hexagon grid


graphing pixels: 100%|██████████| 8811/8811 [00:15<00:00, 575.67it/s]


plotting...


c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_stack.py:76: RuntimeWarning: invalid value encountered in divide
  new_percentages = old_percentages/(old_percentages.sum(axis=1,keepdims=True))
graphing pixels: 100%|██████████| 8811/8811 [00:14<00:00, 614.04it/s]


plotting...


c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_stack.py:114: RuntimeWarning: All-NaN slice encountered
  intensity_skysub = intensity-np.nanmedian(intensity,axis=0)
100%|██████████| 99/99 [02:15<00:00,  1.37s/it]


In [6]:
# transform data to WCS using cross-correlation
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.wcs_datacubes()

calculting 2D cross-correlation shift guesses...


c:\Users\daniel\anaconda3\envs\IFUM\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1217: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,


select 5 WCS stars in order; see popup window
optimizing WCS star coordinates...
performing WCS transforms...


c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:318: RuntimeWarning: divide by zero encountered in scalar divide
  theta = np.degrees(np.arctan(180./(np.pi*(np.sqrt(x__[i]*x__[i]+y__[i]*y__[i])))))


	1156 used 5 reference stars


...

	1168 used 5 reference stars


c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:318: RuntimeWarning: divide by zero encountered in scalar divide
  theta = np.degrees(np.arctan(180./(np.pi*(np.sqrt(x__[i]*x__[i]+y__[i]*y__[i])))))


	1169 used 5 reference stars


c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:318: RuntimeWarning: divide by zero encountered in scalar divide
  theta = np.degrees(np.arctan(180./(np.pi*(np.sqrt(x__[i]*x__[i]+y__[i]*y__[i])))))


	1170 used 5 reference stars
WCS transformations complete


In [4]:
# calibrate intensity for all datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.full_intensity_callibration()

c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_stack.py:324: RuntimeWarning: All-NaN slice encountered
  avg_intensities = np.vstack((avg_intensities,np.nanmedian(npzdata["intensity"],axis=0)))


In [4]:
# stack datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.stack_datacubes()

100%|██████████| 99/99 [55:36<00:00, 33.70s/it]


final data cubes saved!


In [ ]:
from astropy.io import fits
import scipy
file__ = fits.open(r"C:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\spectrabins_int.fits")
data__ = file__[0].data
data__ = data__-np.nanmedian(data__,axis=0)
# data__ = data__-scipy.ndimage.median_filter(data__,size=(1,12))
fits.writeto("test.fits",data__,overwrite=True)
# median_image = scipy.ndimage.median_filter(data__,size=(1,9))